In [1]:
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime as dt
from collections import Counter
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.ar_model import AutoReg
import pmdarima as pm

In [2]:
total_acorn = pd.read_parquet('total_acorn.gzip')
total_usage = pd.read_parquet('total_usage.gzip')

In [3]:
consumption_2012 = total_usage[total_usage.date_time.dt.year == 2012]
consumption_2013 =  total_usage[total_usage.date_time.dt.year == 2013]
consumption_2014 =  total_usage[total_usage.date_time.dt.year == 2014]

In [4]:
# get daily consumption averages to use exog of temp or wind
consumption_2012_daily_mean = consumption_2012.groupby([consumption_2012['date_time'].dt.date, consumption_2012['treated']]).mean()
consumption_2012_daily_mean = consumption_2012_daily_mean.unstack()
consumption_2012_daily_mean.head()
# get daily consumption averages to use exog of temp or wind
consumption_2013_daily_mean = consumption_2013.groupby([consumption_2013['date_time'].dt.date, consumption_2013['treated']]).mean()
consumption_2013_daily_mean = consumption_2013_daily_mean.unstack()
print(consumption_2013_daily_mean.head())

              KWH/hh          
treated        False     True 
date_time                     
2013-01-01  0.250666  0.231662
2013-01-02  0.254633  0.233902
2013-01-03  0.241183  0.223092
2013-01-04  0.240085  0.230233
2013-01-05  0.248812  0.225436


In [5]:
train=consumption_2012_daily_mean[('KWH/hh', False)]
test=consumption_2013_daily_mean[('KWH/hh', False)]

In [3]:
model_no_exog = pm.auto_arima(train, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=7, max_q=7, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=True,   # Seasonality
                      start_P=0,  
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model_no_exog.summary())

NameError: name 'train' is not defined

In [2]:
no_exog_pred= model_no_exog.predict()

NameError: name 'model_no_exog' is not defined

In [8]:
# read in weather data
weather_df = pd.read_csv('weather.csv')
new_dt=[]
for item in weather_df['DT']:
    item = dt.strptime(item, '%Y-%m-%d').strftime('%Y-%m-%d')
    new_dt.append(item)
weather_df['DT']=new_dt
weather_df=weather_df.set_index(weather_df['DT'])
display(weather_df)
weather_2012 = weather_df['T2M']['2012-01-01':'2012-12-31'].reset_index(drop=True)
weather_2013 = weather_df['T2M']['2013-01-01':'2013-12-31'].reset_index(drop=True)

,T2M_RANGE,TS,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M,WS50M_RANGE,WS10M_RANGE,WS50M_MIN,WS50M_MAX,WS10M_MIN,WS10M_MAX,WS50M,WS10M,DT
DT,,,,,,,,,,,,,,,,
2012-01-01,4.28,9.84,9.64,9.64,11.34,7.06,10.33,3.32,3.11,7.94,11.26,5.14,8.25,10.03,7.22,2012-01-01
2012-01-02,1.96,5.59,4.31,4.32,6.84,4.88,5.68,3.60,3.03,7.04,10.64,4.75,7.77,8.88,6.42,2012-01-02
2012-01-03,6.40,7.79,5.98,5.98,11.71,5.31,8.21,13.29,10.69,10.57,23.86,7.79,18.48,17.03,12.91,2012-01-03
2012-01-04,5.19,6.04,4.33,4.33,8.52,3.33,6.21,9.64,7.65,9.52,19.16,6.98,14.62,13.53,10.24,2012-01-04
2012-01-05,4.87,7.78,4.94,4.94,10.17,5.31,8.21,8.42,6.93,12.36,20.78,8.89,15.83,17.45,13.08,2012-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-12-27,5.42,3.02,1.67,1.68,6.80,1.38,2.46,11.61,9.61,7.91,19.52,5.88,15.49,11.93,9.41,2014-12-27
2014-12-28,6.24,0.56,-0.39,-0.38,4.63,-1.61,0.38,5.55,4.59,4.23,9.77,2.84,7.44,7.25,5.10,2014-12-28
2014-12-29,5.80,0.67,-0.08,-0.08,3.98,-1.82,0.16,3.15,2.48,4.16,7.31,3.16,5.64,6.30,4.52,2014-12-29


In [14]:
model_exog = pm.auto_arima(train, weather_2012.to_numpy().reshape(-1,1), start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=7, max_q=7, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=True,   # Seasonality
                      start_P=0,  
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model_exog.summary())

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-2421.464, Time=0.50 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-2376.460, Time=0.14 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-2386.717, Time=0.12 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-2404.916, Time=0.30 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-2378.457, Time=0.12 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-2433.992, Time=0.99 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-2410.175, Time=0.26 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=-2429.695, Time=0.86 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-2426.215, Time=0.56 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-2425.192, Time=0.52 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=-2414.071, Time=0.32 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=-2427.199, Time=1.07 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=-2435.956, Time=0.78 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=-2430.055, Time=0.17 sec
 ARIMA(2,1,0)(0,0,0